In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Batch Processing

여러개의 데이터를 한번의 연산 처리하는 방식  
GPU의 행렬연산로 한번의 메모리 업로드를 통해 연산이 가능 => 속도 향상

**기존의 Linear regression with Gradient Descent를 batch_size 번의 데이터마다 업데이트 하도록 하는 코드 작성**

In [48]:
import pandas as pd
import os

data_path = "/content/drive/MyDrive/nlp-open-tutorial/1일차_배포/colab/LR_dataset"
train_file = "train.csv"
test_file = "test.csv"

df_data = pd.read_csv(os.path.join(data_path, train_file)).dropna()
print(df_data)

data_list_x = list(df_data.loc[:,"x"])
data_list_y = list(df_data.loc[:,"y"])
data_num = len(data_list_x)
print(len(data_list_x))
print(len(data_list_y))

        x          y
0    24.0  21.549452
1    50.0  47.464463
2    15.0  17.218656
3    38.0  36.586398
4    87.0  87.288984
..    ...        ...
695  58.0  58.595006
696  93.0  94.625094
697  82.0  88.603770
698  66.0  63.648685
699  97.0  94.975266

[699 rows x 2 columns]
699
699


In [49]:
def lr_hypo(a,b,x):
  return a * x + b

def lr_loss(y, y_pred):
  return (y - y_pred)**2

In [22]:
def ff_a(a,b,x,y):
  return 2 * (a * x + b - y) * x

def ff_b(a,b,x,y):
  return 2 * (a * x + b - y)

a = 10
b = -1
lr = 0.0001
batch_size = 20
for e in range(100):
  print("epoch ",e)
  loss_sum = 0
  grad_a_sum = 0
  grad_b_sum = 0

  print(a,b)
  for i in range(data_num):
    y_pred = lr_hypo(a, b, data_list_x[i])
    loss_sum += lr_loss(y_pred, data_list_y[i])
    grad_a_sum += ff_a(a, b, data_list_x[i], data_list_y[i])
    grad_b_sum += ff_b(a, b, data_list_x[i], data_list_y[i])
    if (i+1) % batch_size == 0:
      loss_mean = loss_sum / batch_size
      a -= lr * (grad_a_sum / batch_size)
      b -= lr * (grad_b_sum / batch_size)

  print(loss_mean)
  print("\n")

epoch  0
10 -1
119248.39747083852


epoch  1
0.40095292663060067 -1.1907095703968098
555.3229464397517


epoch  2
1.0687348239143672 -1.1766017863190987
13.061018886359296


epoch  3
1.022266709770687 -1.1767436201257735
8.196631583315913


epoch  4
1.0254880557633481 -1.1758948650468795
8.328560400133137


epoch  5
1.0252525802501689 -1.175115761919451
8.318010425322793


epoch  6
1.025257601343816 -1.174332550509654
8.318366873712932


epoch  7
1.0252459013932305 -1.1735503615730096
8.317969975362862


epoch  8
1.0252353746891487 -1.1727688375214502
8.317626173296045


epoch  9
1.0252247763401077 -1.1719880025844465
8.317279335107619


epoch  10
1.0252141929428225 -1.1712078544294853
8.316933364923973


epoch  11
1.0252036184641022 -1.1704283925725087
8.316587989956671


epoch  12
1.0251930533149092 -1.1696496164014294
8.316243228027817


epoch  13
1.0251824974578985 -1.1688715253136368
8.315899076742268


epoch  14
1.0251719508869244 -1.1680941187064275
8.31555553511369


epoch  15


#Linear Regression with Tensor


**Numpy array tensor를 이용한 batch processing 구현**
* numpy.ipynb의 텐서 연산 예시와 ppt를 기반으로 구현

data tensor:  
[[x1, 1],  
 [x2, 1],  
 [x3, 1]]

target tensor:  
[[y1],  
 [y2],  
 [y3]]

param tensor:  
[[a,b]]  

hypo: y = a * x + b * 1   
MSE: (f(x) - y) ^ 2 / n  
MSE의 편미분: 2 * (f(x) - y)f'(x) / n


In [66]:
import numpy as np

def f(param, data):
  return np.dot(data, param.T)

def loss(y, y_pred):
  return (y_pred - y)**2

def ff_a(param, data, y):
  return 2 * (f(param, data) - y) * data[:,:1]

def ff_b(param, data, y):
  return 2 * (f(param, data) - y)

a = 10
b = -1
lr = 0.0001
batch_size = 20
for e in range(100):
  print("epoch ",e)
  print(a,b)
  data_append_list = []
  target_append_list = []
  all_loss = []
  for i in range(data_num):
    data_append_list.append([data_list_x[i], 1])
    target_append_list.append([data_list_y[i]])
    if (i+1) % batch_size == 0:
      param_tensor = np.array([[a,b]])
      target_tensor = np.array(target_append_list)
      data_tensor = np.array(data_append_list)

      y_pred = f(param_tensor, data_tensor)

      all_loss.append(loss(target_tensor, y_pred))
      a -= lr * np.average(ff_a(param_tensor, data_tensor, target_tensor))
      b -= lr * np.average(ff_b(param_tensor, data_tensor, target_tensor))

      data_append_list = []
      target_append_list = []

  print(np.average(all_loss))
  print("\n\n\n")

epoch  0
10 -1
7822.089683585462




epoch  1
1.0058379536856312 -1.1540273917559567
8.348948996282651




epoch  2
1.0058112621988173 -1.152256107800317
8.3480154030247




epoch  3
1.0057846167041782 -1.1504878759489354
8.347085018638406




epoch  4
1.005758017122464 -1.1487226909427228
8.346157832084703




epoch  5
1.0057314633745622 -1.146960547531651
8.345233832362519




epoch  6
1.0057049553814965 -1.1452014404747404
8.344313008508642




epoch  7
1.0056784930644267 -1.14344536454004
8.34339534959761




epoch  8
1.0056520763446481 -1.141692314504615
8.342480844741536




epoch  9
1.0056257051435922 -1.1399422851545291
8.341569483090023




epoch  10
1.0055993793828253 -1.1381952712848322
8.340661253830008




epoch  11
1.0055730989840492 -1.1364512676995417
8.339756146185659




epoch  12
1.0055468638691005 -1.1347102692116275
8.33885414941821




epoch  13
1.0055206739599507 -1.1329722706429983
8.337955252825871




epoch  14
1.0054945291787054 -1.131237266824485
8.337059445

#Multiple Linear Regression with Tensor


**Numpy array tensor를 이용한 batch processing구현**  
* numpy.ipynb의 텐서 연산 예시와 ppt를 기반으로 구현  

data tensor:  
[[x11, x12, 1],  
 [x21, x22, 1],  
 [x31, x32, 1]]

target tensor:  
[[y1],  
 [y2],  
 [y3]]

param tensor:  
[[a,b,c]]  

hypo: y = a * x1 + b * x2 + c   
MSE: (f(x) - y) ^ 2 / n  
MSE의 편미분: 2 * (f(x) - y)f'(x) / n

In [67]:
data_path = "/content/drive/MyDrive/nlp-open-tutorial/1일차_배포/colab/LR_dataset"
train_file = "Marketing_Data.csv"

multi_df_data = pd.read_csv(os.path.join(data_path,train_file))
print(multi_df_data)

x_multi_df = multi_df_data.loc[:, ["youtube","facebook","newspaper"]]
print(x_multi_df)
y_sales = list(multi_df_data.loc[:, "sales"])
print(y_sales)

x_youtube = list(x_multi_df.loc[:, "youtube"])
print(x_youtube)
x_facebook = list(x_multi_df.loc[:, "facebook"])
print(x_facebook)
x_newspaper = list(x_multi_df.loc[:, "newspaper"])
print(x_newspaper)

     youtube  facebook  newspaper  sales
0      84.72     19.20      48.96  12.60
1     351.48     33.96      51.84  25.68
2     135.48     20.88      46.32  14.28
3     116.64      1.80      36.00  11.52
4     318.72     24.00       0.36  20.88
..       ...       ...        ...    ...
166    45.84      4.44      16.56   9.12
167   113.04      5.88       9.72  11.64
168   212.40     11.16       7.68  15.36
169   340.32     50.40      79.44  30.60
170   278.52     10.32      10.44  16.08

[171 rows x 4 columns]
     youtube  facebook  newspaper
0      84.72     19.20      48.96
1     351.48     33.96      51.84
2     135.48     20.88      46.32
3     116.64      1.80      36.00
4     318.72     24.00       0.36
..       ...       ...        ...
166    45.84      4.44      16.56
167   113.04      5.88       9.72
168   212.40     11.16       7.68
169   340.32     50.40      79.44
170   278.52     10.32      10.44

[171 rows x 3 columns]
[12.6, 25.68, 14.28, 11.52, 20.88, 11.4, 15.36, 30.4

In [68]:
import numpy as np

def f(param, data):
  return np.dot(data, param.T)

def loss(y, y_pred):
  return (y_pred - y)**2

def ff_a(param, data, y):
  return 2 * (f(param, data) - y) * data[:,:1]

def ff_b(param, data, y):
  return 2 * (f(param, data) - y) * data[:,1:2]

def ff_c(param, data, y):
  return 2 * (f(param, data) - y) * data[:,2:3]

def ff_d(param, data, y):
  return 2 * (f(param, data) - y) * data[:,3:]

a = 10
b = 3
c = 10
d = 5
lr = 0.00002
batch_size = 20
for e in range(100):
  print("epoch ",e)
  print(a,b)
  data_append_list = []
  target_append_list = []
  all_loss = []
  for i in range(len(x_youtube)):
     data_append_list.append([x_youtube[i], x_facebook[i], x_newspaper[i], 1])
     target_append_list.append([y_sales[i]])

     if (i+1) % batch_size == 0:
      param_tensor = np.array([[a,b,c,d]])
      target_tensor = np.array(target_append_list)
      data_tensor = np.array(data_append_list)

      y_pred = f(param_tensor, data_tensor)

      all_loss.append(loss(target_tensor, y_pred))
      a -= lr * np.average(ff_a(param_tensor, data_tensor, target_tensor))
      b -= lr * np.average(ff_b(param_tensor, data_tensor, target_tensor))
      c -= lr * np.average(ff_c(param_tensor, data_tensor, target_tensor))
      d -= lr * np.average(ff_d(param_tensor, data_tensor, target_tensor))

      data_append_list = []
      target_append_list = []

  print(np.average(all_loss))
  print("\n")

epoch  0
10 3
2448888.7504496477


epoch  1
0.2906118241453983 1.3342651168883009
53087.35774428696


epoch  2
0.015503846876287808 0.6268538323614515
23137.34285849364


epoch  3
0.0344501887847537 0.156845135908323
12752.082474786279


epoch  4
0.053905135998572384 -0.1570201755915658
7211.305116202973


epoch  5
0.06688332746328485 -0.35953205395821164
4175.243049351977


epoch  6
0.07501819291464432 -0.4828223965936032
2493.1478516377965


epoch  7
0.07971551168954041 -0.5502213973003827
1547.3541376296964


epoch  8
0.0820024482628626 -0.578640602030692
1004.4492958921383


epoch  9
0.08262264008716365 -0.580303693696149
683.9913274732841


epoch  10
0.08211250468426687 -0.564013213825995
487.9711823376091


epoch  11
0.08085731623252504 -0.5360777179687696
362.86628938419125


epoch  12
0.0791322498181222 -0.5009901686661317
279.21383890249433


epoch  13
0.07713240635464821 -0.4619240976940293
220.60527589224884


epoch  14
0.07499476566876871 -0.4210962933486796
177.74900282249